In [63]:
import numpy as np
import itertools
import unittest

np.set_printoptions(precision=3)

In [11]:
def regularise(R, k):
    return np.apply_along_axis(lambda r : r / np.abs(r[k]) if r[k] != 0 else r, 1, R)

def check_feasibility(R):
    rhs = R.T[-1]
    return len(list(filter(lambda v : v > 0, rhs))) == 0


In [66]:
def find_xn(R):
    limits = set()
    
    lower_bounds = list(filter(lambda r : r[0] > 0, R))
    if len(lower_bounds) > 0:
        lower_bounds = map(lambda r : r[1] / r[0], lower_bounds)
        lower = max(lower_bounds)
        limits.add(lower)

    upper_bounds = list(filter(lambda r : r[0] < 0, R))
    if len(upper_bounds) > 0:
        upper_bounds = map(lambda r : r[1] / r[0], upper_bounds)
        upper = min(upper_bounds)
        limits.add(upper)
        
    return list(limits)
    
list(find_xn(np.array([
    [1, 4],
    [-1, -5]
])))

    

[4.0, 5.0]

In [50]:
def find_xk(R, xs, k):
    assert(R.shape[1] == (k + 1) + len(xs) + 1)
    xs = np.pad(xs, (k + 1, 1), constant_values=0)
    offset = np.apply_along_axis(sum, 1, R * xs)
    S = np.column_stack([R.T[k], R.T[-1] - offset])
    return find_xn(S)

list(find_xk(np.array([[2, 1, 6]]), [2], 0))

[2.0, inf]

In [51]:
def back_substitute(Rs, n, k):
    if k == n:
        Rs = Rs[:-1]
        k = k - 1
       
    Xs = list(itertools.product([np.inf, -np.inf], repeat=n-k-1))
    
    while k >= 0:
        R = Rs[k]
        Ts = []
        for xs in Xs:
            X_k = list(filter(lambda x : not np.isnan(x), find_xk(R, xs, k)))
            Ts.extend(map(lambda x_k : np.hstack([[x_k], xs]), X_k))
        Xs = Ts
        print(f"back_propagate step {k}:\n{Xs}")
        k -= 1
    
    return np.array(Xs)

# back_substitute([ np.array([[2, 1, 6], [1, 1, -6]]), np.array([[0, 1, 2], [0, -1, -3]]), np.array([[0, -1]]) ], 2, 2)
back_substitute([ np.array([[2, 1, -1, 6], [1, 1, -1, -6]]) ], 3, 0)

back_propagate step 0:
[array([inf, inf, inf]), array([-inf,  inf, -inf]), array([ inf,  inf, -inf]), array([ inf, -inf,  inf]), array([ inf, -inf, -inf])]


/home/ubuntu/.local/lib/python3.6/site-packages/numpy/lib/shape_base.py:379: RuntimeWarning: invalid value encountered in double_scalars
  res = asanyarray(func1d(inarr_view[ind0], *args, **kwargs))
/home/ubuntu/.local/lib/python3.6/site-packages/numpy/lib/shape_base.py:402: RuntimeWarning: invalid value encountered in double_scalars
  buff[ind] = asanyarray(func1d(inarr_view[ind], *args, **kwargs))


array([[ inf,  inf,  inf],
       [-inf,  inf, -inf],
       [ inf,  inf, -inf],
       [ inf, -inf,  inf],
       [ inf, -inf, -inf]])

In [79]:
def generate_constraint(A, b):
    A = np.array(A)
    b = np.array(b)
    n = A.shape[1]
    m = A.shape[0]
    E = np.hstack([A, -A, -np.identity(m), np.array([b]).T])
    E = np.vstack([E, -E])
    V = np.hstack([np.identity(2 * n + m), [[0]] * (2 * n + m)])
    E = np.vstack([E, V])
    return E

generate_constraint([[1, 1], [1, 2]], [6, 2])

array([[ 1.,  1., -1., -1., -1., -0.,  6.],
       [ 1.,  2., -1., -2., -0., -1.,  2.],
       [-1., -1.,  1.,  1.,  1.,  0., -6.],
       [-1., -2.,  1.,  2.,  0.,  1., -2.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.]])

In [102]:
def linprog(c, A, b):
    A = np.array(A)
    b = np.array(b)
    c = np.array(c)
    
    # min c^T x
    # subject to: Ax >= b
    R_0 = generate_constraint(A, b)
    
    Rs = [R_0]
    k = 0
    n = R_0.shape[1] - 1
    while k < n:
        R_k = Rs[k]
        R_k = regularise(R_k, k)
        print(f"step {k}:\n{R_k}")
        
        w = R_k.T[k]
        zeros = []
        ones = []
        minus_ones = []
        for i in range(len(w)):
            if w[i] == 0:
                zeros.append(i)
            elif w[i] == 1:
                ones.append(i)
            else:
                minus_ones.append(i)

        m = len(w)
        if len(zeros) == m: # all 0s
            # mark x_k arbitrary
            Rs.append(R_k)
            k += 1
            continue

        if len(ones) == m or len(minus_ones) == m: # all 1s or -1s
            # we cannot reduce the system
            Xs = back_substitute(Rs, n, k)
            break

        if len(ones) + len(zeros) == m or len(minus_ones) + len(zeros) == m:
            # use the constraints with zero coefficients as reduced system
            S = list(map(lambda i : R_k[i], zeros))
            Rs.append(np.array(S))
            k += 1
            continue

        S = list(map(lambda i : R_k[i], zeros))
        for i in ones:
            for j in minus_ones:
                r = R_k[i] + R_k[j]
                if any(r):
                    S.append(r)
        Rs.append(np.array(S))
        k += 1
    else:
        if not check_feasibility(Rs[-1]):
            return None
        Xs = back_substitute(Rs, n, n)
    
    n = A.shape[1]
    Xs = np.around(Xs, decimals=3)
    Xs = Xs[:, :n] - Xs[:, n: 2*n]
    values = map(lambda xs : (xs, np.sum(xs * c)), Xs)
    values = filter(lambda v : not np.isnan(v[1]), values)
    values = list(values)
    
    print(f"values:\n{values}")
    
    return min(values, key=lambda v: v[1])

In [103]:
A = [[2, 15], [2, 12], [2, 9]]
b = [11, 8, 5]
c = [1, 6]
linprog(c, A, b)

step 0:
[[ 1.   7.5 -1.  -7.5 -0.5 -0.  -0.   5.5]
 [ 1.   6.  -1.  -6.  -0.  -0.5 -0.   4. ]
 [ 1.   4.5 -1.  -4.5 -0.  -0.  -0.5  2.5]
 [-1.  -7.5  1.   7.5  0.5  0.   0.  -5.5]
 [-1.  -6.   1.   6.   0.   0.5  0.  -4. ]
 [-1.  -4.5  1.   4.5  0.   0.   0.5 -2.5]
 [ 1.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   1.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   1.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   1.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   1.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   1.   0. ]]
step 1:
[[ 0.     1.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     1.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     1.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     1.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     1.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     1.     0.   ]
 [ 0.     1.     0.    -1.    -0.333  0.333  0.     1.   ]
 [ 0.     1

(array([-2.,  1.]), 4.0)

In [9]:
class LinProgTest(unittest.TestCase):    
    def test1(self):
        A = [[1], [-1]]
        b = [5, 3]
        c = [1]
        result = linprog(c, A, b)
        self.assertEqual(result[1], -3)
    
    def test2(self):
        A = [[1], [-1]]
        b = [5, -6]
        c = [1]
        result = linprog(c, A, b)
        self.assertEqual(result, None)
    
    def test3(self):
        A = [[1], [-1]]
        b = [5, -6]
        c = [1]
        result = linprog(c, A, b)
        self.assertEqual(result, None)
        
    def test4(self):
        A = [[1, 1], [1, 2]]
        b = [6, 2]
        c = [1, 1]
        result = linprog(c, A, b)
        self.assertEqual(result[1], -np.inf)
        
    def test5(self):
        A = [[1, 1], [1, 2]]
        b = [6, 2]
        c = [1, 1]
        result = linprog(c, A, b)
        self.assertEqual(result[1], -np.inf)
        
unittest.main(argv=[''], verbosity=2, exit=False)

test1 (__main__.LinProgTest) ... ok
test2 (__main__.LinProgTest) ... ok
test3 (__main__.LinProgTest) ... ok
test4 (__main__.LinProgTest) ... /home/ubuntu/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
ok
test5 (__main__.LinProgTest) ... 

step 0:
[[ 1.  5.]
 [-1.  3.]]
back_propagate step 0:
[array([-3.]), array([5.])]
values:
[(array([-3.]), -3.0), (array([5.]), 5.0)]
step 0:
[[ 1.  5.]
 [-1. -6.]]
step 0:
[[ 1.  5.]
 [-1. -6.]]
step 0:
[[1. 1. 6.]
 [1. 2. 2.]]
back_propagate step 0:
[array([-inf,  inf]), array([-inf, -inf]), array([ inf, -inf])]
values:
[(array([-inf, -inf]), -inf)]
step 0:
[[1. 1. 6.]
 [1. 2. 2.]]
back_propagate step 0:
[array([-inf,  inf]), array([-inf, -inf]), array([ inf, -inf])]
values:
[(array([-inf, -inf]), -inf)]


ok

----------------------------------------------------------------------
Ran 5 tests in 0.020s

OK


In [316]:
A = [[2, 15, -5], [2, 12, -4], [2, 9, -3], [0, -1, 0], [0, 0, -1]]
b = [11, 8, 0, 0, 0]
c = [1, 6, -2]
linprog(c, A, b)

step 0:
[[ 1.   7.5 -2.5  5.5]
 [ 1.   6.  -2.   4. ]
 [ 1.   4.5 -1.5  0. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.  -1.   0. ]]
step 1:
[[ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]]
step 2:
[[ 0.  0. -1.  0.]]
back_propagate step 2:
[array([-0.]), array([inf])]
back_propagate step 1:
[array([-0., -0.]), array([inf, -0.]), array([inf, inf])]
back_propagate step 0:
[array([-inf,  -0.,  -0.]), array([ 0., -0., -0.]), array([-inf,  inf,  -0.]), array([-inf,  inf,  inf])]
values:
[(array([-inf,  -0.,  -0.]), -inf), (array([ 0., -0., -0.]), 0.0)]


/home/ubuntu/python-for-finance/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.


(array([-inf,  -0.,  -0.]), -inf)

nan